## start

In [17]:
import os
import cv2
import random
import numpy as np
import torch 
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import torchvision
import torchvision.transforms as tt
import matplotlib.pyplot as plt
from tqdm import tqdm
from torchvision.datasets import ImageFolder
from torch.utils.data import random_split, DataLoader, Dataset
from google.colab import drive
torch.manual_seed(777)
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
if torch.cuda.is_available():
   device = torch.device('cuda')
else:
    device = torch.device('cpu')
print(device)

def to_device(data, device):
    if isinstance(data, (list, tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device

    def __iter__(self):
        for b in self.dl:
            yield to_device(b, self.device)
    
    def __len__(self):
        return len(self.dl)

cuda


In [15]:
## file directory: ICPS_Challenge_Data/label(1-40)/image_name.jpg
class MyDataset(Dataset):
    def __init__(self, path):
        self.path = path
        classes = os.listdir(self.path)
        print(classes)
        cmap = {v:i for i, v in enumerate(classes)}
        self.x = []
        self.y = []
        for label in tqdm(classes):
            #for data in os.listdir(self.path+'/'+label):
            listdir = os.listdir(self.path+'/'+label)
            datalist = random.sample(listdir, 50)
            for data in datalist:
                x = cv2.imread(self.path+'/'+label+'/'+data).astype(np.uint8)
                x = cv2.resize(x,(64, 64)).transpose((2, 0, 1))
                self.x.append(x)
                del x 
                self.y.append(cmap[label])
        del cmap, classes

    def __len__(self):
        return len(self.x)
        
    def __getitem__(self, idx):
        x = torch.from_numpy(self.x[idx])/255
        y = self.y[idx]
        return x, y

In [4]:
dataPath = '/content/drive/Shareddrives/ICPS_Challenge_Data'
dataList = os.listdir(dataPath)
print(dataList)

testData = MyDataset(dataPath)
print(len(testData))

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40']
['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40']


100%|██████████| 40/40 [33:40<00:00, 50.51s/it]

2000 2000
2000


In [7]:
test_dl = DataLoader(testData)

## model class

In [8]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

def conv_block(in_channel, out_channel, pool=False):
    layers = [nn.Conv2d(in_channel, out_channel, kernel_size=3, padding=1, stride=1), nn.ReLU(True)]
    if pool:
        layers.append(nn.MaxPool2d(2,2))
    return nn.Sequential(*layers)

class ImageClassfication(nn.Module):
    def train_step(self, batch):
        images, classes = batch
        out = self(images)          #Generate prediction
        loss = F.cross_entropy(out, classes)   #Calculate loss
        return loss

    def validation_step(self, batch):
        images, classes = batch
        out = self(images)                  
        loss = F.cross_entropy(out, classes)
        acc = accuracy(out, classes)            #Calculate Accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}

    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses

        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies

        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], val_loss: {:.4f}, val_acc: {:.4f}".format(epoch, result['val_loss'], result['val_acc']))

class VGG19(ImageClassfication):
    def __init__(self, num_classes):
        super().__init__()
        self.network = nn.Sequential(
            conv_block(3, 64),
            conv_block(64, 64, True),   #64x64x64

            conv_block(64, 128),
            conv_block(128, 128, True), #128x32x32

            conv_block(128, 256),
            conv_block(256, 256),
            conv_block(256, 256),
            conv_block(256, 256, True), #256x16x16

            conv_block(256, 512),
            conv_block(512, 512),
            conv_block(512, 512),
            conv_block(512, 512, True), #512x8x8

            conv_block(512, 512),
            conv_block(512, 512),
            conv_block(512, 512),
            conv_block(512, 512, True), #512x4x4

            #classifier
            nn.Sequential(
                nn.Flatten(), 
                nn.Linear(512 * 2 * 2, 1024),
                nn.ReLU(True),
                nn.Dropout(),
                nn.Linear(1024, 1024),
                nn.ReLU(True),
                nn.Dropout(),
                nn.Linear(1024, num_classes)
                )
        )
        self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

    def forward(self, xb):
         return self.network(xb)

In [9]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

def conv_block(in_channel, out_channel, pool=False):
    layers = [nn.Conv2d(in_channel, out_channel, kernel_size=3, padding=1, stride=1), nn.ReLU(True)]
    if pool:
        layers.append(nn.MaxPool2d(2,2))
    return nn.Sequential(*layers)

class ImageClassfication(nn.Module):
    def train_step(self, batch):
        images, classes = batch
        out = self(images)          #Generate prediction
        loss = F.cross_entropy(out, classes)   #Calculate loss
        return loss

    def validation_step(self, batch):
        images, classes = batch
        out = self(images)                  
        loss = F.cross_entropy(out, classes)
        acc = accuracy(out, classes)            #Calculate Accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}

    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses

        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies

        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], val_loss: {:.4f}, val_acc: {:.4f}".format(epoch, result['val_loss'], result['val_acc']))

class VGG19(ImageClassfication):
    def __init__(self, num_classes):
        super().__init__()
        self.network = nn.Sequential(
            conv_block(3, 64),
            conv_block(64, 64, True),   #64x64x64

            conv_block(64, 128),
            conv_block(128, 128, True), #128x32x32

            conv_block(128, 256),
            conv_block(256, 256),
            conv_block(256, 256),
            conv_block(256, 256, True), #256x16x16

            conv_block(256, 512),
            conv_block(512, 512),
            conv_block(512, 512),
            conv_block(512, 512, True), #512x8x8

            conv_block(512, 512),
            conv_block(512, 512),
            conv_block(512, 512),
            conv_block(512, 512, True), #512x4x4

            #classifier
            nn.Sequential(
                nn.Flatten(), 
                nn.Linear(512 * 2 * 2, 1024),
                nn.ReLU(True),
                nn.Dropout(),
                nn.Linear(1024, 1024),
                nn.ReLU(True),
                nn.Dropout(),
                nn.Linear(1024, num_classes)
                )
        )
        self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

    def forward(self, xb):
         return self.network(xb)

## Test

In [12]:
!ls
os.chdir('/content/drive/MyDrive')

'Colab Notebooks'	  icps_challenge.onnx   kinect_vgg19.pth
 curt_challenge_1.ipynb   icpsChallenge.onnx
'ICPS (1).ipynb'	  kinect.pth


In [16]:
test_model = torch.load('./kinect_vgg19.pth')
test_model.to(device)
test_dl = DeviceDataLoader(test_dl, device)

yes = 0
no = 0
for images, labels in tqdm(test_dl):
    out = test_model(images)
    _, pred = torch.max(out, dim=1)
    for i in range(len(pred)):
        if  pred[i] == labels[i]:
            yes = yes + 1
        else:
            no = no +1

total = yes + no
correct_percent = yes/total * 100
print("\ntesting result:", correct_percent, "%")

2000 2000


  1%|          | 13/2000 [00:00<00:15, 125.80it/s]

2000 2000


100%|██████████| 2000/2000 [00:08<00:00, 235.21it/s]


testing result: 97.95 %
